## Import libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pytz
from scipy import stats

## Define util functions

In [ ]:
def extract_attribute_from_line(attribute, line):
    match = re.search(rf"{attribute}: (\d+)", line)
    return int(match.group(1)) if match else np.nan


def timestamp_str_to_datetime(timestamp_str):
    # Convert the timestamp string to a datetime object
    return pytz.timezone('Europe/Paris').localize(datetime.datetime.strptime(timestamp_str, "%Y-%m-%d %H:%M:%S"))


def timestamp_str_to_unix(timestamp_str):
    # Convert the timestamp string to a datetime object
    dt_obj = datetime.datetime.strptime(timestamp_str, "%Y-%m-%d %H:%M:%S").replace(tzinfo=pytz.timezone('Europe/Paris'))

    # Convert the datetime object to a Unix timestamp
    unix_timestamp = int(dt_obj.timestamp())

    return unix_timestamp

def unix_to_datetime(unix_timestamp):
    return datetime.datetime.fromtimestamp(unix_timestamp, tz=pytz.timezone('Europe/Paris'))


def separate_system_on_and_off_per_day(df, date):
    # Define the time range for grouping (9am to 5pm)
    start_time = datetime.time(9, 0)
    end_time = datetime.time(17, 0)

    # Initialize a dictionary to store the groups for each day
    daily_groups = {}

    # Iterate through the timestamps and group them accordingly
    for dt in df.timestamp:  
        date_key = dt.date()  # Use the date as the key for the dictionary

        if start_time <= dt.time() <= end_time:

            tens_number = (dt.minute // 10) + 1  # Calculate the interval

            # Create a new list for the day if it doesn't exist in the dictionary
            if date_key not in daily_groups:
                daily_groups[date_key] = {"system_on": [], "system_off": []}

            # Append the timestamp to the appropriate group based on the quarter
            if tens_number in [1, 3, 5]:
                daily_groups[date_key]["system_on"].append(dt)
            elif tens_number in [2, 4, 6]:
                daily_groups[date_key]["system_off"].append(dt)

    single_day_system_on_timestamps = daily_groups[date]["system_on"]
    single_day_system_off_timestamps = daily_groups[date]["system_off"]

    single_day_system_on_df = df[df.timestamp.isin(single_day_system_on_timestamps)]
    single_day_system_off_df = df[df.timestamp.isin(single_day_system_off_timestamps)]
  
    return single_day_system_on_df, single_day_system_off_df
 

def read_txt_to_df(file_path):
    #file_path: example "long-monitorlog_2023-07-28.txt"
    file = open(file_path, "r")
    lines = file.readlines()

    data = []
    buffer_lines = []
    for line in lines:
        if "veml7700_lux" in line:
            buffer_lines.append(line)
        elif "as7262_temp" in line:
            buffer_lines.append(line)
        elif "veml6070_uv" in line:
            buffer_lines.append(line)
        else:
            continue

        if len(buffer_lines) == 3:
            first_timestamp = int(re.search(r"\['\d+:\d+:\d+', '(\d+)'\]", buffer_lines[0]).group(1))
            last_timestamp = int(re.search(r"\['\d+:\d+:\d+', '(\d+)'\]", buffer_lines[2]).group(1))
            if last_timestamp - first_timestamp > 2:
                print("Something is wrong: ", first_timestamp, last_timestamp, last_timestamp - first_timestamp)

            combined_line = " ".join(buffer_lines)
            timestamp = int(re.search(r"\['\d+:\d+:\d+', '(\d+)'\]", combined_line).group(1))
            timestamp_wrong = extract_attribute_from_line("Timestamp", combined_line)
            node_id = extract_attribute_from_line("node_id", combined_line)
            veml7700_lux = extract_attribute_from_line("veml7700_lux", combined_line)
            sgp40_voc = extract_attribute_from_line("sgp40_voc", combined_line)
            shtc3_temperature = extract_attribute_from_line("shtc3_temperature", combined_line)
            shtc3_humidity = extract_attribute_from_line("shtc3_humidity", combined_line)
            as7262_temp = extract_attribute_from_line("as7262_temp", combined_line)
            as7262_violet = extract_attribute_from_line("as7262_violet", combined_line)
            as7262_blue = extract_attribute_from_line("as7262_blue", combined_line)
            as7262_green = extract_attribute_from_line("as7262_green", combined_line)
            as7262_yellow = extract_attribute_from_line("as7262_yellow", combined_line)
            as7262_orange = extract_attribute_from_line("as7262_orange", combined_line)
            as7262_red = extract_attribute_from_line("as7262_red", combined_line)
            veml6070_uv = extract_attribute_from_line("veml6070_uv", combined_line)
            si1145_lux = extract_attribute_from_line("si1145_lux", combined_line)
            si1145_infrared = extract_attribute_from_line("si1145_infrared", combined_line)
            si1145_uv = extract_attribute_from_line("si1145_uv", combined_line)

            # break
            single_row = [timestamp,node_id, veml7700_lux, sgp40_voc, shtc3_temperature, shtc3_humidity,
                    as7262_temp, as7262_violet, as7262_blue, as7262_green, as7262_yellow, as7262_orange, 
                    as7262_red, veml6070_uv, si1145_lux, si1145_infrared, si1145_uv]
            
            data.append(single_row)

            buffer_lines.clear()

    columns =   ['timestamp','node_id', 'veml7700_lux', 'sgp40_voc', 'shtc3_temperature', 'shtc3_humidity',
                'as7262_temp', 'as7262_violet', 'as7262_blue', 'as7262_green', 'as7262_yellow', 'as7262_orange', 
                'as7262_red', 'veml6070_uv', 'si1145_lux', 'si1145_infrared', 'si1145_uv']
    df =  pd.DataFrame(data, columns = columns)
    return df

def assign_intervals_to_column(df, target_column, start_time, end_time):
    """
    params:
    df: a dataframe which should have a column timestamp of type datetime.
    target_column: 
    """
    paris_tz = pytz.timezone('Europe/Paris')

    # Define the time range for the plot (from 9 am to 5 pm)
    start_datetime = timestamp_str_to_datetime(start_time)#"e.g. ,2023-07-30 09:00:00"
    end_datetime = timestamp_str_to_datetime(end_time)#e.g.,("2023-07-30 17:00:00")

    target_column_df = df[['timestamp', target_column]].copy()
    
    # Group data into 10-minute intervals
    target_column_df['interval'] = pd.cut(target_column_df['timestamp'], pd.date_range(start_datetime, end_datetime, freq='10T'))

    # Convert interval labels to desired format
    target_column_df['interval'] = target_column_df['interval'].apply(lambda x: f"{x.mid.strftime('%H:%M')}")

    return target_column_df

    
def clean_sensor_data(df, switch_setting = True):
    df = df.copy()

    # Convert Unix timestamps to datetime objects
    df['timestamp'] = df['timestamp'].apply(lambda x: unix_to_datetime(x))

    if switch_setting:
        # remove the data in the first minute of each interval
        df = df[df['timestamp'].dt.minute % 10 != 0]

    # remove unreasonable lux values and cct values
    print(f'Exclude {df[df.veml7700_lux > 3000].shape[0]} rows because of > 3000 lx.')
    cleaned_df = df[df.veml7700_lux <= 3000]


    return cleaned_df


def add_cct_column(df):
    
    data = df.copy()

    V = data.as7262_violet
    G = data.as7262_green
    R = data.as7262_red

    X = -0.14282 * R + 1.54924 * G + -0.95641 * V
    Y = -0.32466 * R + 1.57837 * G + -0.73191 * V
    Z = -0.68202 * R + 0.77073 * G + 0.56332 * V

    #Step 2: calculate the chromaticity coordinates
    x = X/(X + Y + Z)
    y = Y/(X + Y + Z)

    #Step 3: McCamy’s formula (third version)
    #https://onlinelibrary.wiley.com/doi/epdf/10.1002/col.5080170211
    n = (x - 0.3320) / (0.1858 - y)
    cct = 449 * n**3 + 3525 * n**2 + 6823.3 * n + 5520.33

    data.loc[:,'cct'] = cct
    
    return data


 

def barplot(system_on_data, system_off_data, date_str,  target_column, lower_hline = 500, higher_hline = 1000):
    #date_str: example: 2023-07-31
    grouped_system_on_data = assign_intervals_to_column(system_on_data, target_column, f"{date_str} 09:00:00", f"{date_str} 17:00:00")
    grouped_system_off_data = assign_intervals_to_column(system_off_data, target_column, f"{date_str} 09:00:00", f"{date_str} 17:00:00")
    
    # Create the bar plot with error bars using seaborn
    barplot_fig, ax = plt.subplots(figsize=(10, 6))
    sns.barplot(x='interval', y = target_column, data=grouped_system_on_data, ax=ax,color = '#404040', label = "Smart Lighting Control System Switched On")
    sns.barplot(x='interval', y = target_column, data=grouped_system_off_data, ax=ax, color = '#A0A0A0', label = "No Lamp Switched On")


    y_value_label = "Illuminance (in lux)"
    plt.ylim(0,1500)
    if target_column == 'cct':
        y_value_label = "CCT (in Kelvin)"
        plt.ylim(0,8000)
    # Set plot labels and title
    ax.set(xlabel='Time of the day', ylabel=f'Average {y_value_label}') 
    # title=f'{y_value_label} from 9am to 5pm on day {date_str}')

    # Plot horizontal lines at y=500 and y=1000
    ax.axhline(y=lower_hline, color='black', linestyle='dashed')
    ax.axhline(y=higher_hline, color='black', linestyle='dashed')

    ax.legend(loc='upper right')

    # Set the x-ticks to display one point every hour
    hourly_tick_labels = ['09:05', '10:05', '11:05', '12:05', '13:05', '14:05', '15:05', '16:05' ]
    
    plt.xticks([i for i in range(0, 48, 6)], hourly_tick_labels)

    return barplot_fig, grouped_system_on_data, grouped_system_off_data

def report_temp_humidity(df, start_time, end_time):

    paris_tz = pytz.timezone('Europe/Paris')

    # Define the time range for the plot (from 9 am to 5 pm)
    start_datetime = timestamp_str_to_datetime(start_time)#"e.g. ,2023-07-30 09:00:00"
    end_datetime = timestamp_str_to_datetime(end_time)#e.g.,("2023-07-30 17:00:00")


    df = df.copy()

    df = df[(df.timestamp >= start_datetime) & (df.timestamp < end_datetime)]
    temp_mean = df.shtc3_temperature.mean()
    temp_std = df.shtc3_temperature.std()

    humidity_mean = df.shtc3_humidity.mean()
    humidity_std = df.shtc3_humidity.std()

    return temp_mean, temp_std, humidity_mean, humidity_std






## Read files and separate data when system was on and off

In [ ]:
data = read_txt_to_df("data/monitorlog_2023-08-15.txt")
data = add_cct_column(data)
cleaned_sensor_data = clean_sensor_data(data)
report_temp_humidity(cleaned_sensor_data, "2023-08-15 09:00:00", "2023-08-15 17:00:00")

In [ ]:

system_on_data_0815, system_off_data_0815 = separate_system_on_and_off_per_day(cleaned_sensor_data, datetime.date(2023, 8, 15))


In [ ]:
system_off_data_0815.cct.mean()

In [ ]:
system_off_data_0815.cct.std()

## Plot lineplot for illuminance

In [ ]:
barplot_fig, grouped_system_on_data, grouped_system_off_data = barplot(system_on_data_0815, system_off_data_0815, "2023-08-15", "veml7700_lux")

## Compare illuminance and CCT


In [ ]:
def check_normality(df, column):
    """
    if n<= 50:
        shapiro test
    else:
        D'Agostino's K^2 test

    Parameters:
    df: a dataframe.
    column: the column that whose normality needs to be checked

    Returns:
    num_points: the number of data points
    is_normality: a boolean value indicating whether the data is normally distributed
    """
    num_points = df[~df[column].isna()].shape[0] 
    if num_points <= 50:
       normality_test = stats.shapiro(df[column])
    else:
        normality_test = stats.normaltest(df[column])
    if normality_test.pvalue <= 0.05:
        return num_points, True
    else:
        return num_points, False
    


def get_mean_std(df, column):
    """
    Parameters:
    df: a dataframe.
    column: the column that whose normality needs to be checked

    Returns:
    mean: the mean of the column
    std: the standard deviation of the column
    """
    data = df[column]
    return data.mean(), data.std()

def get_median_iqr(df, column):
    # remove nan numbers first
    data = df[~df[column].isna()][column]
    median = data.median()
    q1 = np.percentile(data, 25, interpolation='midpoint')
    q3 = np.percentile(data, 75, interpolation='midpoint')
    iqr = q3 - q1
    return median, iqr


def get_column_stats(df, column):

    n, is_normal = check_normality(df, column)
    if is_normal:
        mean, std = get_mean_std(df, column)
        return n, is_normal, mean, std
    else:
        median, iqr = get_median_iqr(df, column)
        return n, is_normal, median, iqr
    

def welchs_t(group1, group2):
    t_stat, p_value = stats.ttest_ind(group1, group2, equal_var=False)
    return t_stat, p_value

def comparison_table(system_on_data, system_off_data, target_column):
    results = []
    results.append(get_column_stats(system_on_data, target_column))
    results.append(get_column_stats(system_off_data, target_column))
   
    results.append(welchs_t(system_on_data[target_column], system_off_data[target_column]))

    return results

def calculate_distance_from_range(x, lower, upper):
    if x < lower:
        return max(lower - x, 0)
    elif x > upper:
        return max(x - upper, 0)
    else:
        return 0

def distance_comparison_table(system_on_data, system_off_data, target_column, target_lower, target_higher):
    
    system_on_data = system_on_data.copy()
    system_off_data = system_off_data.copy()

    #append distances to the dataframes
    system_on_data['distance'] = system_on_data[target_column].apply(calculate_distance_from_range, args=(target_lower, target_higher))
    system_off_data['distance'] = system_off_data[target_column].apply(calculate_distance_from_range, args=(target_lower, target_higher))
    
    results = []
    results.append(get_column_stats(system_on_data, "distance"))
    results.append(get_column_stats(system_off_data, "distance"))

    results.append(welchs_t(system_on_data["distance"], system_off_data["distance"]))
    
    return results

In [ ]:
distance_comparison_table(system_on_data_0815, system_off_data_0815, "veml7700_lux", 500, 1000)

In [ ]:
comparison_table(system_on_data_0815, system_off_data_0815, "veml7700_lux")

## Dynamic lighting figure

In [ ]:

def dynamic_barplot(dynamic_data, date_str,  target_column, lower_hline = 500, higher_hline = 1000):
    #date_str: example: 2023-07-31

    grouped_dynamic_data = assign_intervals_to_column(dynamic_data, target_column, f"{date_str} 09:00:00", f"{date_str} 17:00:00")

    # Create the bar plot with error bars using seaborn
    barplot_fig, ax = plt.subplots(figsize=(10, 6))
    sns.barplot(x='interval', y = target_column, data=grouped_dynamic_data, ax=ax,color = '#808080', label = "Smart Lighting Control System Switched On")
  
    y_value_label = "Illuminance (in lux)"
    # 9, 12, 13, 14, 16, 17, 19
    hours = [0, 18, 24, 30, 42, 48]#[0, 12, 30, 36, 42, 54, 60, 72]
    hours = [hour - 0.5 for hour in hours] #five minutes move from the center
    print(hours)
    target_values = [1000, 500, 500, 1000, 500, 500] #[500, 1000, 500, 500, 1000, 500, 500, 500] 
    variance = 50
    plt.ylim(0,1500)
    if target_column == 'cct':
        y_value_label = "CCT (in Kelvin)"
        target_values = [5000, 4000, 4000, 5000, 4000, 4000, 4000] #[5000, 4000, 4000, 5000, 4000, 4000]
        variance = 300
        plt.ylim(0,15000)
    # Set plot labels and title
    ax.set(xlabel='Time of the day', ylabel=f'Average {y_value_label}') 
    # title=f'{y_value_label} from 9am to 5pm on day {date_str}')

    # Plot horizontal lines at y=500 and y=1000
    ax.axhline(y=lower_hline, color='black', linestyle='dashed')
    ax.axhline(y=higher_hline, color='black', linestyle='dashed')

    ax.plot(hours, target_values, label='Target line', color='black')

    # Create upper and lower shadow lines
    upper_shadow = [val + variance for val in target_values]
    lower_shadow = [val - variance for val in target_values]

    # Fill the shadow regions
    ax.fill_between(hours, lower_shadow, upper_shadow, color='#404040', alpha=0.2, label='Target area')



    ax.legend()

    # Set the x-ticks to display one point every hour
    hourly_tick_labels = ['09:05', '10:05', '11:05', '12:05', '13:05', '14:05', '15:05', '16:05']
    
    plt.xticks([i for i in range(0, 48, 6)], hourly_tick_labels)
    

    return barplot_fig, grouped_dynamic_data



In [ ]:


dynamic_data = read_txt_to_df("data/dynamic_monitorlog_2023-08-12.txt")
dynamic_data = add_cct_column(dynamic_data)

cleaned_dynamic_data = clean_sensor_data(dynamic_data, switch_setting = False)
report_temp_humidity(cleaned_dynamic_data, "2023-08-13 09:00:00", "2023-08-13 17:00:00")

In [ ]:
figure_lux_0812, df_lux_0812= dynamic_barplot(cleaned_dynamic_data, "2023-08-13", "veml7700_lux", lower_hline = 500, higher_hline = 1000)